# Gemini Plugin

> Plugin implementation for Google Gemini API transcription

In [ ]:
#| default_exp plugin

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import logging
import tempfile
from pathlib import Path
from typing import Dict, Any, Optional, List, Union, Tuple
import re

import numpy as np

from fastcore.basics import patch

try:
    from google import genai
    from google.genai import types
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False

try:
    from cjm_ffmpeg_utils.core import FFMPEG_AVAILABLE
    from cjm_ffmpeg_utils.audio import downsample_audio
except ImportError:
    FFMPEG_AVAILABLE = False
    
# Import domain-specific plugin interface from migrated system
from cjm_transcription_plugin_system.plugin_interface import TranscriptionPlugin
from cjm_transcription_plugin_system.core import AudioData, TranscriptionResult

In [ ]:
#| export
class GeminiPlugin(TranscriptionPlugin):
    """Google Gemini API transcription plugin."""
    
    # Default audio-capable models (can be overridden)
    DEFAULT_AUDIO_MODELS = [
        "gemini-2.5-flash",
        "gemini-2.5-flash-preview-05-20",
        "gemini-2.5-pro",
        "gemini-2.5-pro-preview-05-06",
        "gemini-2.0-flash",
        "gemini-2.0-flash-exp",
        "gemini-1.5-flash",
        "gemini-1.5-flash-latest",
        "gemini-1.5-pro",
        "gemini-1.5-pro-latest",
    ]
    
    def __init__(self):
        """Initialize the Gemini plugin with default configuration."""
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.client = None
        self.available_models = []
        self.model_token_limits = {}  # Store model name -> output_token_limit mapping
        self.uploaded_files = []  # Track uploaded files for cleanup
    
    @property
    def name(
        self
    ) -> str:  # Returns the plugin name identifier
        """Return the plugin name identifier."""
        return "gemini"
    
    @property
    def version(
        self
    ) -> str:  # Returns the plugin version string
        """Return the plugin version string."""
        return "1.0.0"
    
    @property
    def supported_formats(
        self
    ) -> List[str]:  # Returns list of supported audio formats
        """Return list of supported audio file formats."""
        return ["wav", "mp3", "aiff", "aac", "ogg", "flac"]

    @staticmethod
    def get_config_schema(
        current_model: str="gemini-2.5-flash",
        max_tokens: int=65536,
        available_models: List[str]=None
    ) -> Dict[str, Any]:  # Returns JSON schema for configuration validation
        """Return configuration schema for Gemini."""
        
        if not available_models:
            available_models=GeminiPlugin.DEFAULT_AUDIO_MODELS
        
        return {
            "$schema": "http://json-schema.org/draft-07/schema#",
            "type": "object",
            "title": "Gemini Configuration",
            "properties": {
                "model": {
                    "type": "string",
                    "default": "gemini-2.5-flash",
                    "description": "Gemini model to use for transcription",
                    "enum": available_models
                },
                "api_key": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Google API key (defaults to GEMINI_API_KEY env var)"
                },
                "prompt": {
                    "type": "string",
                    "default": "Generate a transcription of the audio, only extract speech and ignore background audio.",
                    "description": "Prompt for transcription"
                },
                "temperature": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 2.0,
                    "default": 0.0,
                    "description": "Sampling temperature"
                },
                "top_p": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.95,
                    "description": "Top-p sampling parameter"
                },
                "max_output_tokens": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": max_tokens,
                    "default": max_tokens,
                    "description": f"Maximum number of output tokens (max for {current_model}: {max_tokens})"
                },
                "seed": {
                    "type": ["integer", "null"],
                    "default": None,
                    "description": "Random seed for reproducibility"
                },
                "response_mime_type": {
                    "type": "string",
                    "enum": ["text/plain", "application/json"],
                    "default": "text/plain",
                    "description": "Response MIME type"
                },
                "downsample_audio": {
                    "type": "boolean",
                    "default": False,
                    "description": "Downsample audio before uploading (requires ffmpeg)"
                },
                "downsample_rate": {
                    "type": "integer",
                    "enum": [8000, 16000, 22050, 44100],
                    "default": 16000,
                    "description": "Target sample rate for downsampling"
                },
                "downsample_channels": {
                    "type": "integer",
                    "enum": [1, 2],
                    "default": 1,
                    "description": "Number of audio channels (1=mono, 2=stereo)"
                },
                "safety_settings": {
                    "type": "string",
                    "enum": ["OFF", "BLOCK_NONE", "BLOCK_FEW", "BLOCK_SOME", "BLOCK_MOST"],
                    "default": "OFF",
                    "description": "Safety filter threshold"
                },
                "auto_refresh_models": {
                    "type": "boolean",
                    "default": True,
                    "description": "Automatically refresh available models list"
                },
                "model_filter": {
                    "type": "array",
                    "items": {"type": "string"},
                    "default": [],
                    "description": "Keywords to exclude from model names (e.g., ['tts', 'image'])"
                },
                "use_file_upload": {
                    "type": "boolean",
                    "default": False,
                    "description": "Upload audio files to Gemini API instead of embedding in request"
                },
                "use_streaming": {
                    "type": "boolean",
                    "default": False,
                    "description": "Use streaming response for transcription"
                },
                "delete_uploaded_files": {
                    "type": "boolean",
                    "default": True,
                    "description": "Delete uploaded files after transcription"
                }
            },
            "required": ["model"],
            "additionalProperties": False
        }

    def get_current_config(
        self
    ) -> Dict[str, Any]:  # Returns the merged configuration dictionary
        """Return current configuration."""
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}

    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None  # Configuration dictionary to override defaults
    ) -> None:
        """Initialize the plugin with configuration."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        # Initialize client
        try:
            api_key = self._get_api_key()
            self.client = genai.Client(api_key=api_key)
            
            # Refresh available models if enabled
            if self.config.get("auto_refresh_models", True):
                self.available_models = self._refresh_available_models()
                # Update schema with actual available models
                schema = self.get_config_schema()
                schema["properties"]["model"]["enum"] = self.available_models
            else:
                self.available_models = self.DEFAULT_AUDIO_MODELS
            
            # Update max_output_tokens based on selected model's limit
            self._update_max_tokens_for_model(self.config['model'])
                
            self.logger.info(f"Initialized Gemini plugin with model '{self.config['model']}'")
            
        except Exception as e:
            self.logger.error(f"Failed to initialize Gemini client: {e}")
            raise

    def execute(
        self,
        audio: Union[AudioData, str, Path],  # Audio data object or path to audio file
        **kwargs # Additional arguments to override config
    ) -> TranscriptionResult:  # Returns transcription result object
        """Transcribe audio using Gemini."""
        if not self.client:
            raise RuntimeError("Plugin not initialized. Call initialize() first.")
        
        # Check if model is being overridden at execution time
        if "model" in kwargs and kwargs["model"] != self.config.get("model"):
            self._update_max_tokens_for_model(kwargs["model"])
        
        # Prepare audio file
        audio_path, temp_created = self._prepare_audio(audio)
        uploaded_file = None
        
        try:
            # Merge runtime kwargs with config
            exec_config = {**self.config, **kwargs}
            
            # Decide whether to upload file or embed in request
            if exec_config.get("use_file_upload", False):
                # Upload audio file to Gemini API
                uploaded_file = self._upload_audio_file(audio_path)
                audio_content = uploaded_file
            else:
                # Read audio file and embed in request (existing behavior)
                with open(audio_path, 'rb') as f:
                    audio_bytes = f.read()
                
                # Determine MIME type
                suffix = audio_path.suffix.lower()
                mime_map = {
                    '.wav': 'audio/wav',
                    '.mp3': 'audio/mp3',
                    '.aiff': 'audio/aiff',
                    '.aac': 'audio/aac',
                    '.ogg': 'audio/ogg',
                    '.flac': 'audio/flac'
                }
                mime_type = mime_map.get(suffix, 'audio/wav')
                
                # Create audio content part
                audio_content = types.Part.from_bytes(
                    data=audio_bytes,
                    mime_type=mime_type
                )
            
            # Prepare generation config
            generate_config = types.GenerateContentConfig(
                response_mime_type=exec_config["response_mime_type"],
                temperature=exec_config["temperature"],
                top_p=exec_config["top_p"],
                max_output_tokens=exec_config["max_output_tokens"],
                seed=exec_config.get("seed"),
                safety_settings=[
                    types.SafetySetting(
                        category="HARM_CATEGORY_HATE_SPEECH",
                        threshold=exec_config["safety_settings"]
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold=exec_config["safety_settings"]
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                        threshold=exec_config["safety_settings"]
                    ),
                    types.SafetySetting(
                        category="HARM_CATEGORY_HARASSMENT",
                        threshold=exec_config["safety_settings"]
                    )
                ]
            )
            
            # Prepare contents
            prompt = exec_config["prompt"]
            contents = [prompt, audio_content]
            
            # Generate transcription
            self.logger.info(f"Transcribing with Gemini model: {exec_config['model']} (max_tokens: {exec_config['max_output_tokens']})")
            
            # Use streaming or regular generation
            if exec_config.get("use_streaming", False):
                # Use streaming response
                transcribed_text = ""
                for chunk in self.client.models.generate_content_stream(
                    model=exec_config["model"],
                    contents=contents,
                    config=generate_config
                ):
                    if hasattr(chunk, 'text'):
                        transcribed_text += chunk.text
                        # Could optionally yield chunks here for real-time processing
                self.logger.info("Streaming transcription completed")
            else:
                # Use regular response (existing behavior)
                response = self.client.models.generate_content(
                    model=exec_config["model"],
                    contents=contents,
                    config=generate_config
                )
                transcribed_text = response.text if hasattr(response, 'text') else str(response)
            
            # Create result
            result = TranscriptionResult(
                text=transcribed_text.strip(),
                confidence=None,  # Gemini doesn't provide confidence scores
                segments=None,  # Gemini doesn't provide segments by default
                metadata={
                    "model": exec_config["model"],
                    "temperature": exec_config["temperature"],
                    "top_p": exec_config["top_p"],
                    "max_output_tokens": exec_config["max_output_tokens"],
                    "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                    "use_file_upload": exec_config.get("use_file_upload", False),
                    "use_streaming": exec_config.get("use_streaming", False)
                }
            )
            
            self.logger.info(f"Transcription completed: {len(transcribed_text.split())} words")
            return result
            
        finally:
            # Clean up uploaded file if configured to do so
            if uploaded_file and exec_config.get("delete_uploaded_files", True):
                try:
                    self._delete_uploaded_file(uploaded_file.name)
                    # Remove from tracking list
                    self.uploaded_files = [f for f in self.uploaded_files if f.name != uploaded_file.name]
                except Exception as e:
                    self.logger.warning(f"Failed to cleanup uploaded file: {e}")
            
            # Clean up temporary file
            if temp_created:
                try:
                    audio_path.unlink()
                except Exception:
                    pass

    def is_available(
        self
    ) -> bool:  # Returns True if the Gemini API is available
        """Check if Gemini API is available."""
        return GEMINI_AVAILABLE

In [ ]:
#| export
@patch
def _get_api_key(
    self:GeminiPlugin
) -> str:  # Returns the API key string
    """Get API key from config or environment."""
    api_key = self.config.get("api_key")
    if not api_key:
        api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("No API key provided. Set GEMINI_API_KEY environment variable or provide api_key in config")
    return api_key

@patch
def _refresh_available_models(
    self:GeminiPlugin
) -> List[str]:  # Returns list of available model names
    """Fetch and filter available models from Gemini API."""
    try:
        if not self.client:
            return self.DEFAULT_AUDIO_MODELS
        
        # Get all models that support content generation
        all_models = list(self.client.models.list())
        gen_models = [model for model in all_models if 'generateContent' in model.supported_actions]
        
        # Extract model names and apply filters
        model_filter = self.config.get("model_filter", [])
        if not model_filter:
            model_filter = ['tts', 'image', 'learn']  # Default exclusions
        
        filtered_names = []
        self.model_token_limits = {}  # Reset token limits
        
        for model in gen_models:
            model_name = model.name.removeprefix('models/')
            # Skip if any filter keyword is in the model name
            if not any(keyword in model_name.lower() for keyword in model_filter):
                filtered_names.append(model_name)
                # Store the output token limit for this model
                self.model_token_limits[model_name] = model.output_token_limit
        
        # Sort with newest/best models first
        filtered_names.sort(reverse=True)
        
        self.logger.info(f"Found {len(filtered_names)} audio-capable models")
        return filtered_names if filtered_names else self.DEFAULT_AUDIO_MODELS
        
    except Exception as e:
        self.logger.warning(f"Could not fetch models from API: {e}. Using defaults.")
        return self.DEFAULT_AUDIO_MODELS

@patch
def _update_max_tokens_for_model(
    self:GeminiPlugin,
    model_name: str  # Model name to update tokens for
) -> None:
    """Update max_output_tokens config based on the model's token limit."""
    if model_name in self.model_token_limits:
        token_limit = self.model_token_limits[model_name]
        # Update the config with the model's actual limit
        self.config["max_output_tokens"] = token_limit
        self.logger.info(f"Updated max_output_tokens to {token_limit} for model '{model_name}'")
    else:
        # Use a sensible default if we don't have the limit
        default_limit = 65536  # Common default for newer models
        self.config["max_output_tokens"] = default_limit
        self.logger.info(f"Using default max_output_tokens of {default_limit} for model '{model_name}'")

In [ ]:
#| export
@patch
def update_config(
    self:GeminiPlugin,
    config: Dict[str, Any]  # New configuration values
) -> None:
    """Update plugin configuration, adjusting max_tokens if model changes."""
    old_model = self.config.get("model")
    
    # Update config through parent class or directly
    self.config.update(config)
    
    # If model changed, update max_output_tokens
    new_model = self.config.get("model")
    if new_model and new_model != old_model:
        self._update_max_tokens_for_model(new_model)

@patch
def _prepare_audio(
    self:GeminiPlugin,
    audio: Union[AudioData, str, Path]  # Audio data object or path to audio file
) -> Tuple[Path, bool]:  # Returns tuple of (processed audio path, whether temp file was created)
    """Prepare audio file for upload.
    
    Returns:
        Tuple of (audio_path, is_temp_file)
    """
    temp_created = False
    
    if isinstance(audio, (str, Path)):
        audio_path = Path(audio)
    elif isinstance(audio, AudioData):
        # Save AudioData to temporary file
        import soundfile as sf
        temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        
        audio_array = audio.samples
        # Convert to mono if stereo
        if audio_array.ndim > 1:
            audio_array = audio_array.mean(axis=1)
        
        # Ensure float32 and normalized
        if audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        if audio_array.max() > 1.0:
            audio_array = audio_array / np.abs(audio_array).max()
        
        sf.write(temp_file.name, audio_array, audio.sample_rate)
        audio_path = Path(temp_file.name)
        temp_created = True
    else:
        raise ValueError(f"Unsupported audio input type: {type(audio)}")
    
    # Optionally downsample audio
    if self.config.get("downsample_audio", False) and FFMPEG_AVAILABLE:
        try:
            downsampled = audio_path.with_stem(f"{audio_path.stem}_downsampled")
            downsample_audio(
                audio_path,
                downsampled,
                sample_rate=self.config.get("downsample_rate", 16000),
                channels=self.config.get("downsample_channels", 1)
            )
            
            # Clean up original temp file if created
            if temp_created:
                audio_path.unlink()
            
            audio_path = downsampled
            temp_created = True
            self.logger.info(f"Downsampled audio to {self.config['downsample_rate']}Hz")
            
        except Exception as e:
            self.logger.warning(f"Failed to downsample audio: {e}")
    
    return audio_path, temp_created

@patch
def _upload_audio_file(
    self:GeminiPlugin,
    audio_path: Path  # Path to audio file to upload
) -> Any:  # Returns uploaded file object
    """Upload audio file to Gemini API.
    
    Returns:
        Uploaded file object
    """
    try:
        self.logger.info(f"Uploading audio file: {audio_path}")
        uploaded_file = self.client.files.upload(file=audio_path)
        self.uploaded_files.append(uploaded_file)  # Track for cleanup
        self.logger.info(f"Successfully uploaded file: {uploaded_file.name}")
        return uploaded_file
    except Exception as e:
        self.logger.error(f"Failed to upload audio file: {e}")
        raise

@patch
def _delete_uploaded_file(
    self:GeminiPlugin,
    file_name: str  # Name of file to delete
) -> None:
    """Delete an uploaded file from Gemini API."""
    try:
        self.client.files.delete(name=file_name)
        self.logger.info(f"Deleted uploaded file: {file_name}")
    except Exception as e:
        self.logger.warning(f"Failed to delete uploaded file {file_name}: {e}")

In [ ]:
#| export
@patch
def cleanup(
    self:GeminiPlugin
) -> None:
    """Clean up resources."""
    # Clean up any remaining uploaded files
    if self.config.get("delete_uploaded_files", True):
        for uploaded_file in self.uploaded_files:
            try:
                self._delete_uploaded_file(uploaded_file.name)
            except Exception as e:
                self.logger.warning(f"Failed to delete file during cleanup: {e}")
    
    self.uploaded_files = []
    self.client = None
    self.logger.info("Cleanup completed")

@patch
def get_available_models(
    self:GeminiPlugin
) -> List[str]:  # Returns list of available model names
    """Get list of available audio-capable models."""
    if self.config.get("auto_refresh_models", True) and self.client:
        self.available_models = self._refresh_available_models()
    return self.available_models

@patch
def get_model_info(
    self:GeminiPlugin,
    model_name: Optional[str] = None  # Model name to get info for, defaults to current model
) -> Dict[str, Any]:  # Returns dict with model information
    """Get information about a specific model including token limits."""
    if model_name is None:
        model_name = self.config.get("model", "gemini-2.5-flash")
    
    return {
        "name": model_name,
        "output_token_limit": self.model_token_limits.get(model_name, 65536),
        "current_max_output_tokens": self.config.get("max_output_tokens", 65536)
    }

In [ ]:
#| export
from typing import Generator

@patch
def supports_streaming(
    self:GeminiPlugin
) -> bool:  # Returns True if streaming is supported
    """Check if this plugin supports streaming transcription.
    
    Returns:
        bool: True, as Gemini supports streaming transcription
    """
    return True

@patch  
def execute_stream(
    self:GeminiPlugin,
    audio: Union[AudioData, str, Path],  # Audio data object or path to audio file
    **kwargs  # Additional arguments to override config
) -> Generator[str, None, TranscriptionResult]:  # Yields text chunks, returns final result
    """Stream transcription results chunk by chunk.
    
    This method streams transcription chunks in real-time as they are generated
    by the Gemini API.
    
    Args:
        audio: Audio data or path to audio file
        **kwargs: Additional plugin-specific parameters
        
    Yields:
        str: Partial transcription text chunks as they become available
        
    Returns:
        TranscriptionResult: Final complete transcription with metadata
        
    Example:
        >>> # Stream transcription chunks in real-time
        >>> for chunk in plugin.execute_stream(audio_file):
        ...     print(chunk, end="", flush=True)
    """
    if not self.client:
        raise RuntimeError("Plugin not initialized. Call initialize() first.")
    
    # Force streaming mode in config
    kwargs['use_streaming'] = True
    
    # Check if model is being overridden at execution time
    if "model" in kwargs and kwargs["model"] != self.config.get("model"):
        self._update_max_tokens_for_model(kwargs["model"])
    
    # Prepare audio file
    audio_path, temp_created = self._prepare_audio(audio)
    uploaded_file = None
    
    try:
        # Merge runtime kwargs with config
        exec_config = {**self.config, **kwargs}
        
        # Decide whether to upload file or embed in request
        if exec_config.get("use_file_upload", False):
            # Upload audio file to Gemini API
            uploaded_file = self._upload_audio_file(audio_path)
            audio_content = uploaded_file
        else:
            # Read audio file and embed in request
            with open(audio_path, 'rb') as f:
                audio_bytes = f.read()
            
            # Determine MIME type
            suffix = audio_path.suffix.lower()
            mime_map = {
                '.wav': 'audio/wav',
                '.mp3': 'audio/mp3',
                '.aiff': 'audio/aiff',
                '.aac': 'audio/aac',
                '.ogg': 'audio/ogg',
                '.flac': 'audio/flac'
            }
            mime_type = mime_map.get(suffix, 'audio/wav')
            
            # Create audio content part
            audio_content = types.Part.from_bytes(
                data=audio_bytes,
                mime_type=mime_type
            )
        
        # Prepare generation config
        generate_config = types.GenerateContentConfig(
            response_mime_type=exec_config["response_mime_type"],
            temperature=exec_config["temperature"],
            top_p=exec_config["top_p"],
            max_output_tokens=exec_config["max_output_tokens"],
            seed=exec_config.get("seed"),
            safety_settings=[
                types.SafetySetting(
                    category="HARM_CATEGORY_HATE_SPEECH",
                    threshold=exec_config["safety_settings"]
                ),
                types.SafetySetting(
                    category="HARM_CATEGORY_DANGEROUS_CONTENT",
                    threshold=exec_config["safety_settings"]
                ),
                types.SafetySetting(
                    category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
                    threshold=exec_config["safety_settings"]
                ),
                types.SafetySetting(
                    category="HARM_CATEGORY_HARASSMENT",
                    threshold=exec_config["safety_settings"]
                )
            ]
        )
        
        # Prepare contents
        prompt = exec_config["prompt"]
        contents = [prompt, audio_content]
        
        # Generate transcription with streaming
        self.logger.info(f"Streaming transcription with Gemini model: {exec_config['model']} (max_tokens: {exec_config['max_output_tokens']})")
        
        transcribed_text = ""
        chunks_yielded = 0
        
        # Stream chunks as they arrive
        for chunk in self.client.models.generate_content_stream(
            model=exec_config["model"],
            contents=contents,
            config=generate_config
        ):
            if hasattr(chunk, 'text'):
                chunk_text = chunk.text
                transcribed_text += chunk_text
                chunks_yielded += 1
                yield chunk_text  # Yield each chunk in real-time
        
        self.logger.info(f"Streaming completed: {chunks_yielded} chunks, {len(transcribed_text.split())} words")
        
        # Return final result
        return TranscriptionResult(
            text=transcribed_text.strip(),
            confidence=None,  # Gemini doesn't provide confidence scores
            segments=None,  # Gemini doesn't provide segments by default
            metadata={
                "model": exec_config["model"],
                "temperature": exec_config["temperature"],
                "top_p": exec_config["top_p"],
                "max_output_tokens": exec_config["max_output_tokens"],
                "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                "use_file_upload": exec_config.get("use_file_upload", False),
                "use_streaming": True,
                "streaming_chunks": chunks_yielded
            }
        )
        
    finally:
        # Clean up uploaded file if configured to do so
        if uploaded_file and exec_config.get("delete_uploaded_files", True):
            try:
                self._delete_uploaded_file(uploaded_file.name)
                # Remove from tracking list
                self.uploaded_files = [f for f in self.uploaded_files if f.name != uploaded_file.name]
            except Exception as e:
                self.logger.warning(f"Failed to cleanup uploaded file: {e}")
        
        # Clean up temporary file
        if temp_created:
            try:
                audio_path.unlink()
            except Exception:
                pass

## Testing the Plugin

In [ ]:
# Test basic functionality
plugin = GeminiPlugin()

# Check availability
print(f"Gemini available: {plugin.is_available()}")
print(f"Plugin name: {plugin.name}")
print(f"Plugin version: {plugin.version}")
print(f"Supported formats: {plugin.supported_formats}")

Gemini available: True
Plugin name: gemini
Plugin version: 1.0.0
Supported formats: ['wav', 'mp3', 'aiff', 'aac', 'ogg', 'flac']


In [ ]:
# Test configuration schema
schema = plugin.get_config_schema()
print("Configuration properties:")
for prop, details in list(schema["properties"].items())[:5]:
    print(f"  {prop}: {details.get('description', 'No description')}")

Configuration properties:
  model: Gemini model to use for transcription
  api_key: Google API key (defaults to GEMINI_API_KEY env var)
  prompt: Prompt for transcription
  temperature: Sampling temperature
  top_p: Top-p sampling parameter


In [ ]:
# Test initialization (requires API key)
if os.environ.get("GEMINI_API_KEY"):
    plugin.initialize({"model": "gemini-2.5-flash"})
    print(f"Initialized with model: {plugin.config['model']}")
    
    # Get available models
    models = plugin.get_available_models()
    print(f"\nFound {len(models)} available models")
    print("Top 5 models:")
    for model in models[:5]:
        print(f"  - {model}")
else:
    print("Set GEMINI_API_KEY environment variable to test initialization")

Initialized with model: gemini-2.5-flash

Found 34 available models
Top 5 models:
  - gemma-3n-e4b-it
  - gemma-3n-e2b-it
  - gemma-3-4b-it
  - gemma-3-27b-it
  - gemma-3-1b-it


## Testing Dynamic Token Limits

Test that max_output_tokens is dynamically updated based on the selected model's output_token_limit.

In [ ]:
# Test dynamic token limit updates
if os.environ.get("GEMINI_API_KEY"):
    # Initialize plugin
    plugin = GeminiPlugin()
    plugin.initialize({"model": "gemini-2.5-flash"})
    
    # Check token limits for different models
    print("Token limits for different models:")
    print("-" * 50)
    
    # Display token limits that were discovered
    for model_name in list(plugin.model_token_limits.keys())[:5]:
        token_limit = plugin.model_token_limits[model_name]
        print(f"{model_name}: {token_limit:,} tokens")
    
    print("\nCurrent configuration:")
    print(f"Model: {plugin.config['model']}")
    print(f"Max output tokens: {plugin.config['max_output_tokens']:,}")
    
    # Get model info
    model_info = plugin.get_model_info()
    print(f"\nModel info for {model_info['name']}:")
    print(f"  Output token limit: {model_info['output_token_limit']:,}")
    print(f"  Current max_output_tokens: {model_info['current_max_output_tokens']:,}")
else:
    print("Set GEMINI_API_KEY environment variable to test token limits")

Token limits for different models:
--------------------------------------------------
gemini-2.5-pro-preview-03-25: 65,536 tokens
gemini-2.5-flash-preview-05-20: 65,536 tokens
gemini-2.5-flash: 65,536 tokens
gemini-2.5-flash-lite-preview-06-17: 65,536 tokens
gemini-2.5-pro-preview-05-06: 65,536 tokens

Current configuration:
Model: gemini-2.5-flash
Max output tokens: 65,536

Model info for gemini-2.5-flash:
  Output token limit: 65,536
  Current max_output_tokens: 65,536


In [ ]:
# Test switching models and automatic token limit update
if os.environ.get("GEMINI_API_KEY"):
    # Switch to a different model
    print("Testing model switching and token limit updates:")
    print("-" * 50)
    
    test_models = ["gemini-2.5-flash", "gemini-1.5-pro", "gemini-2.0-flash"]
    
    for model_name in test_models:
        if model_name in plugin.model_token_limits:
            # Update configuration with new model
            plugin.update_config({"model": model_name})
            
            print(f"\nSwitched to model: {model_name}")
            print(f"  Token limit: {plugin.model_token_limits[model_name]:,}")
            print(f"  Config max_output_tokens: {plugin.config['max_output_tokens']:,}")
            
            # Verify schema is updated
            schema = plugin.get_config_schema()
            max_tokens_prop = schema["properties"]["max_output_tokens"]
            print(f"  Schema maximum: {max_tokens_prop['maximum']:,}")
            print(f"  Schema default: {max_tokens_prop['default']:,}")
else:
    print("Set GEMINI_API_KEY environment variable to test model switching")

Testing model switching and token limit updates:
--------------------------------------------------

Switched to model: gemini-2.5-flash
  Token limit: 65,536
  Config max_output_tokens: 65,536
  Schema maximum: 65,536
  Schema default: 65,536

Switched to model: gemini-2.0-flash
  Token limit: 8,192
  Config max_output_tokens: 8,192
  Schema maximum: 65,536
  Schema default: 65,536


In [ ]:
# Test execution with runtime model override
if os.environ.get("GEMINI_API_KEY"):
    print("Testing runtime model override:")
    print("-" * 50)
    
    # Create test audio
    import numpy as np
    from cjm_transcription_plugin_system.core import AudioData
    
    test_audio = AudioData(
        samples=np.random.randn(16000).astype(np.float32) * 0.1,
        sample_rate=16000,
        duration=1.0,
        filepath=None,
        metadata={}
    )
    
    # Current model and token limit
    print(f"Current model: {plugin.config['model']}")
    print(f"Current max_output_tokens: {plugin.config['max_output_tokens']:,}")
    
    # Execute with a different model at runtime
    override_model = "gemini-2.0-flash" if plugin.config['model'] != "gemini-2.0-flash" else "gemini-2.5-flash"
    
    if override_model in plugin.model_token_limits:
        print(f"\nExecuting with override model: {override_model}")
        print(f"Expected token limit: {plugin.model_token_limits[override_model]:,}")
        
        try:
            result = plugin.execute(
                test_audio,
                model=override_model,
                prompt="This is a test audio signal."
            )
            
            print(f"\nTranscription metadata:")
            print(f"  Model used: {result.metadata['model']}")
            print(f"  Max output tokens: {result.metadata['max_output_tokens']:,}")
            
            # Check if config was updated
            print(f"\nConfig after execution:")
            print(f"  Model: {plugin.config['model']}")
            print(f"  Max output tokens: {plugin.config['max_output_tokens']:,}")
            
        except Exception as e:
            print(f"Execution error (expected for random audio): {e}")
else:
    print("Set GEMINI_API_KEY environment variable to test runtime override")

Testing runtime model override:
--------------------------------------------------
Current model: gemini-2.0-flash
Current max_output_tokens: 8,192

Executing with override model: gemini-2.5-flash
Expected token limit: 65,536

Transcription metadata:
  Model used: gemini-2.5-flash
  Max output tokens: 65,536

Config after execution:
  Model: gemini-2.0-flash
  Max output tokens: 65,536


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()